# 1A. Importing dependencies

In [33]:
import mesa
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

# 1B. Helper Functions

In [34]:
def get_distance(pos_1, pos_2):
    '''
    Getting the euclidian distance by using the pytaghorean theorem

    used in trade.move()
    '''

    x1, x2, = pos_1
    y1, y2 = pos_2

    dx = x1 - x2
    dy = y1 - y2

    return math.sqrt(dx**2 + dy**2)

# 2. Resource Classes

In [35]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - Contains an amount of sugar
    - Grows one amount of sugar at each turn
    '''

    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):
        '''
        Sugar Growth function:
        Adds one unit of sugar each step until max_sugar is met
        '''
        self.amount = min([self.max_sugar, self.amount+1])
        

In [36]:
class Spice(mesa.Agent):
    '''
    Spice:
    - Contains an amount of spice
    - Grows one amnount of spice at each turn
    '''

    def __init__(self, unique_id, model, pos, max_spice):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self):
        '''
        Spice growth function:
        Adds one unit of spice each step until max_spice is met
        '''
        self.amount = min([self.max_spice, self.amount+1])

# 3. Trader Class

In [43]:
class Trader(mesa.Agent):
    '''
    Trader:
    - Has a metabolism for sugar and spice
    - Harvests and trade sugar and spice to survive and thrive
    '''

    def __init__(self, unique_id, model, pos, moore=False, sugar=0,
                 spice=0,metabolism_sugar=0,metabolism_spice=0,
                 vision=0):

        super().__init__(unique_id,model)
        self.pos = pos
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision

    ################################
    ### RESOURCE CHECK FUNCTIONS ###
    ################################

    def get_sugar(self, pos):
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if type(agent) is Sugar:
                return agent
        return None

    def get_sugar_amount(self, pos):
        '''
        Helper function of calculate_welfare()
        Get sugar amount of cells around agent
        '''
        sugar_patch = self.get_sugar(pos)
        if sugar_patch:
            return sugar_patch.amount
        return 0
    
    def get_spice(self, pos):
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if type(agent) is Spice:
                return agent
        return None
    
    def get_spice_amount(self, pos):
        '''
        Helpter function of calculate_welfare()
        Get spice amount of cells around agent
        '''
        spice_patch = self.get_spice(pos)
        if spice_patch:
            return spice_patch.amount
        return 0


    def is_occupied_by_other(self, pos):
        '''
        Helper function part of self.move
        Check if a cell is occupied
        '''
        if pos == self.pos:
            # agent position is considered unoccupied as agent can stay there
            return False
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for a in this_cell:
            # see if occupied by another agent
            if isinstance(a, Trader):
                return True
        return False
    

    ##################################
    ### TRADER BEHAVIOUR FUNCTIONS ###
    ##################################

    def calculate_welfare(self, sugar, spice):
        '''
        Helper function of self.move

        '''
        # Calculate total resources that the agent has
        m_total =  self.metabolism_sugar + self.metabolism_spice

        # Cobb douglas function:
        ## https://inomics.com/terms/cobb-douglas-production-function-1456726

        return sugar**(self.metabolism_sugar/m_total) * spice**(self.metabolism_spice/m_total)

    def move(self):
        '''
        Function for trader agent to identify optimal move for each step
        1 - Identify all possible moves
        2 - Determine which move maximise welfare
        3 - Find closest best option
        4 - Move
        '''
        
        # 1 - Identify all possible moves

        ## This gives us a list of ALL possible locations our Trade can go
        ## This is based on their "vision", and takes into consideration cells that are already occupied
        neighbors = [i
                      for i in self.model.grid.get_neighborhood(
                        self.pos, self.moore, True, self.vision
                      ) if not self.is_occupied_by_other(i)]
        
        # 2 - Determine which move maximises welfare
        welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos), 
                self.spice + self.get_spice_amount(pos)) 
            for pos in neighbors
        ]
        
        # 3 - Find closest best option

        ## find the highest welfare in welfares 
        max_welfare = max(welfares)

        ## get the index of max welfare cells
        candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare, rel_tol=1e-02)]
        
        ## convert index to positions of those cells
        candidates = [neighbors[i] for i in candidate_indices]
        
        min_dist = min(get_distance(self.pos, pos) for pos in candidates)

        final_candidates = [ pos for pos in candidates
                            if math.isclose(get_distance(self.pos, pos), min_dist, rel_tol=1e-02)

                            ]
        
        self.random.shuffle(final_candidates)

        # 4. Move agents
        ## get the shuffled list of final candidates and move to the first one
        self.model.grid.move_agent(self, final_candidates[0])

        print(min_dist, final_candidates, final_candidates[0])

# 4. Model Class

In [44]:
class SugarscapeG1mt(mesa.Model):
    '''
    A model class to manage sugarscape with traders (G1mt)
    from: Growing artificial societies 1996
    '''
    def __init__(
            self, width=50,height=50, 
            initial_population=200,
            endowment_min=25,endowment_max=50, 
            metabolism_min=1,metabolism_max=5,
            vision_min=1,vision_max=5
                 ):

        # Inititiate width and heigth of sugarscape
        self.width = width
        self.height = height
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max


        # Initiate scheduler
        self.schedule = mesa.time.RandomActivationByType(self)

        
        # Initiate mesa grid class
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

        # Read in landscape file from supplementary material
        sugar_distribution = np.genfromtxt('sugar-map.txt')
        spice_distribution = np.flip(sugar_distribution, 1)
        
        agent_id = 0

        for _,x,y in self.grid.coord_iter():

            max_sugar = sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(agent_id, self, (x, y), max_sugar)
                self.grid.place_agent(sugar, (x, y))
                self.schedule.add(sugar)
                agent_id += 1
        
        for _,x,y in self.grid.coord_iter():

            max_spice = spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(agent_id, self, (x, y), max_spice)
                self.grid.place_agent(spice, (x, y))
                self.schedule.add(spice)
                agent_id += 1

        for i in range(self.initial_population):
            # get each agent position
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)

            # Giver agents initial endowment
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))

            # giver agents initial metabolism
            metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))

            # Give agent vision
            vision = int(self.random.uniform(self.vision_min, self.vision_max))

            # Create a trader object
            trader = Trader(agent_id, 
                            self,
                            (x,y),
                            moore = False,
                            sugar = sugar,
                            spice = spice,
                            metabolism_sugar = metabolism_sugar,
                            metabolism_spice = metabolism_spice,
                            vision = vision
                            )
            
            # place agent
            self.grid.place_agent(trader, (x,y))
            self.schedule.add(trader)
            
            

            agent_id += 1
    
    def step(self):

        '''
        Unique step function to combine staged activation of sugar and spice
        and then randomly activates traders
        '''

        # step sugar agents
        for sugar in self.schedule.agents_by_type[Sugar].values():
            sugar.step()

        # step spice agents
        for spice in self.schedule.agents_by_type[Spice].values():
            spice.step()

        # step trader agents
        ## to account for agent death and removal, we need a separate data structure to iterate
        trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()
        
        self.schedule.steps += 1 # important for data collector to track the number of steps

    def run_model(self, step_count=1000):

        for i in range(step_count):
            self.step()

# 5. Run Sugarscape

In [45]:
model = SugarscapeG1mt()
model.run_model(step_count=5)

53.075418038862395 [(46, 10), (45, 9), (44, 8), (43, 7)] (46, 10)
25.612496949731394 [(44, 28), (48, 32), (45, 29), (47, 31), (46, 30)] (44, 28)
18.601075237738275 [(18, 29), (19, 30)] (18, 29)
19.209372712298546 [(31, 43), (28, 40), (30, 42), (29, 41)] (31, 43)
52.3450093132096 [(36, 0), (38, 2), (37, 1)] (36, 0)
2.0 [(37, 37), (35, 35)] (37, 37)
13.601470508735444 [(2, 10), (3, 11), (4, 12), (1, 9)] (2, 10)
7.211102550927978 [(45, 41), (47, 43), (46, 42)] (45, 41)
59.413803110051795 [(43, 2), (45, 4)] (43, 2)
37.48332962798263 [(29, 3), (30, 4)] (29, 3)
43.93176527297759 [(13, 42), (17, 46), (14, 43), (15, 44), (16, 45)] (13, 42)
22.67156809750927 [(22, 37), (21, 36), (23, 38)] (22, 37)
1.0 [(21, 21), (20, 20)] (21, 21)
26.90724809414742 [(44, 26)] (44, 26)
15.811388300841896 [(21, 12), (19, 10), (20, 11)] (21, 12)
17.029386365926403 [(14, 3), (12, 1)] (14, 3)
6.4031242374328485 [(24, 20)] (24, 20)
50.99019513592785 [(3, 37), (2, 36), (1, 35), (4, 38)] (3, 37)
26.248809496813376 [(48